# DReS-FL Experiment 

- K = 1
- T = 1
- input dimension = 2
- linear regression
- w = [1 1]
- finite field size = 5
- inputs are selected from {0, 1}
- no relationship between x and y
- the gradient is supposed to be revealed at the end of each training round

Basically, our objective function is $l = (y - xw)^2$ and our gradient is $g = -2(xy - x^2w)$ 

In [1]:
import numpy as np
from lcc.polynomials import LCCPoly, InterpolatedPoly
from mutual_information.estimators.neural.benchmark import neural_estimator_benchmark

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
prime = 5
data_range = 2
para_param = 1 # K
priv_param = 1 # T

num_of_samples = 5000
beta_arr = [0, 1]
alpha_arr = [2, 3, 4] # number of clients = 3
weight = np.asarray([[1], [1]])

In [3]:
def create_encoded_dataset(encoded_data_pol, data_size, encoded_label_pol, alpha):
    encoded_data = np.empty((len(encoded_data_pol), *data_size)) 
    encoded_label = np.empty((len(encoded_label_pol), 1))
    for idx, (data_pol, label_pol) in enumerate(zip(encoded_data_pol, encoded_label_pol)):
        encoded_data[idx] = data_pol(alpha)
        encoded_label[idx][0] = label_pol(alpha)
    return encoded_data, encoded_label

def calculate_gradient_samplewise(encoded_data, encoded_label, curr_weight, p):
    # g = -2(xy - x^2w)
    gradient = np.empty((encoded_data.shape[0], *curr_weight.shape))
    for idx in range(encoded_data.shape[0]):
        data, label = encoded_data[idx], encoded_label[idx]
        gradient[idx] = (label - (data.T @ curr_weight)[0][0]) % p
        gradient[idx] = (data * gradient[idx]) % p
        gradient[idx] = (-2 * gradient[idx]) % p
    gradient = gradient.astype(int)
    return gradient

In [4]:
secret_data = np.random.randint(low=0, high=2, size=(num_of_samples, 2, 1))
secret_label = np.random.randint(low=0, high=2, size=(num_of_samples, 1))
encoded_secret_data_pol = [LCCPoly(beta_arr, [x], para_param, priv_param, prime, size=(2, 1)) for x in secret_data]
encoded_secret_label_pol = [LCCPoly(beta_arr, [x[0]], para_param, priv_param, prime) for x in secret_label]

In [5]:
# client 0
client_0_encoded_data, client_0_encoded_label = create_encoded_dataset(encoded_secret_data_pol, (2, 1), encoded_secret_label_pol, alpha_arr[0])
client_0_encoded_gradient = calculate_gradient_samplewise(client_0_encoded_data, client_0_encoded_label, weight, prime)

# client 1
client_1_encoded_data, client_1_encoded_label = create_encoded_dataset(encoded_secret_data_pol, (2, 1), encoded_secret_label_pol, alpha_arr[1])
client_1_encoded_gradient = calculate_gradient_samplewise(client_1_encoded_data, client_1_encoded_label, weight, prime)

# client 2
client_2_encoded_data, client_2_encoded_label = create_encoded_dataset(encoded_secret_data_pol, (2, 1), encoded_secret_label_pol, alpha_arr[2])
client_2_encoded_gradient = calculate_gradient_samplewise(client_2_encoded_data, client_2_encoded_label, weight, prime)

In [6]:
revealed_gradients = np.empty(client_0_encoded_data.shape)
revealed_random = np.empty(client_0_encoded_gradient.shape)
revealed_poly_constructed_coeff = np.empty((client_0_encoded_gradient.shape[0], 2 * (para_param + priv_param - 1) + 1, client_0_encoded_gradient.shape[1], client_0_encoded_gradient.shape[2]))
for gradient_idx in range(client_0_encoded_gradient.shape[0]):
    revealed_poly = InterpolatedPoly([client_0_encoded_gradient[gradient_idx], client_1_encoded_gradient[gradient_idx], client_2_encoded_gradient[gradient_idx]], alpha_arr, prime)
    revealed_gradients[gradient_idx] = revealed_poly(beta_arr[0])
    revealed_random[gradient_idx] = revealed_poly(beta_arr[1])
    revealed_poly_constructed_coeff[gradient_idx] = revealed_poly.coefficients

In [7]:
# create dataset
resulting_dataset = np.concatenate([secret_data.reshape(num_of_samples, -1), secret_label, revealed_gradients.reshape(num_of_samples, -1), revealed_random.reshape(num_of_samples, -1), revealed_poly_constructed_coeff.reshape(num_of_samples, -1)], axis=1)

In [9]:
estimators_all, results_all = neural_estimator_benchmark(resulting_dataset[:, :3], resulting_dataset[:, 7:])

 18%|█████████▎                                           | 1750/10000 [02:09<10:11, 13.50step/s, test=1.87, train=1.93]


Donker Varadhan estimator: 1.88


 22%|███████████▋                                        | 2250/10000 [00:06<00:23, 331.96step/s, test=1.90, train=2.26]


MINE estimator: 1.93


 18%|█████████▎                                           | 1750/10000 [02:09<10:11, 13.48step/s, test=1.90, train=1.91]


InfoNCE estimator: 1.90


 12%|██████▋                                              | 1250/10000 [01:33<10:51, 13.43step/s, test=1.88, train=1.92]

NWJ estimator: 1.88


In [10]:
estimators_revealed_gradient, results_revealed_gradient = neural_estimator_benchmark(resulting_dataset[:, :3], resulting_dataset[:, 3:5])

 25%|█████████████▎                                       | 2500/10000 [03:16<09:50, 12.69step/s, test=1.38, train=1.36]


Donker Varadhan estimator: 1.39


 12%|██████▌                                             | 1250/10000 [00:05<00:41, 213.22step/s, test=1.41, train=1.51]


MINE estimator: 1.44


 10%|█████▎                                               | 1000/10000 [01:36<14:24, 10.41step/s, test=1.38, train=1.32]


InfoNCE estimator: 1.38


 25%|█████████████▎                                       | 2500/10000 [04:09<12:28, 10.02step/s, test=1.38, train=1.36]

NWJ estimator: 1.38
